[View in Colaboratory](https://colab.research.google.com/github/davidkant/mai/blob/master/tutorial/5_1_Hello_Genetic_Algorithm.ipynb)

# 5.1 Hello Genetic Algorithm
In this notebook we'll walk throgh the general structure of a genetic algorithm and learn how to code up the basic elements.

## Setup

In [0]:
# install external libraries
!pip install -q git+https://github.com/davidkant/mai#egg=mai;
!pip install -q pretty_midi
!pip install -q pyfluidsynth
!apt-get -qq update
!apt-get -qq install -y libfluidsynth1

In [2]:
# imports
import mai
import random

Using TensorFlow backend.


# The Genetic Algorithm Model
The course package `mai` includes a simple genetic algorithm  `GeneticAlgorithm()`. Think of it as an empty shell of a genetic algorithm. It contains the general sequence of steps a GA follows — initialize population, evaluate fitness, reproduce, and repeat — but allows you to customize the GA to your particular application by substituting your own functions for each step. In this notebok we'll take a look at the default functions. **Note:** This is the code `GeneticAlgorithm()` will run if you don't customize anything!

First, create an empty `GeneticAlgothim()` model to work with.

In [0]:
ga = mai.genalg.GeneticAlgorithm()

## 1. Initialize random population
The first step of the GA is to generate an initial population of random individuals. The individuals of this population are *genotypes*. A genotype is a coded representation of the individual. We will represent each genotype as a list of numbers.  

The GA model generates the initial population by calling the function `random_individual` which returns a random genotype. The cell below shows the default code that the GA model uses to generate a random genotype.  By default, a random individual is a list of 10 random numbers, each either a `0` or `1`.

In [0]:
def random_individual():
    """Generate a random genotype."""
    return [random.randrange(0,2) for i in range(10)]

Now let's see what it does. In the cell below we initialize a random population and view it. `GeneticAlgorithm()` stores its current popuation in a variable  named `population`.

In [5]:
# generate an initial population
ga.initialize_population()
# let's see it
ga.population

[[0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 1, 0, 1, 0, 0, 0],
 [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
 [1, 0, 1, 0, 1, 1, 0, 0, 1, 0],
 [1, 1, 0, 1, 1, 0, 0, 1, 1, 1],
 [1, 0, 1, 0, 0, 1, 0, 0, 1, 0],
 [1, 1, 1, 0, 1, 1, 1, 0, 0, 0]]

## 2. Convert genotype to phenotype
The second step of the GA is to convert all of the genotypes to *phenotypes*. The phenotype is the expression of the individual's genotype — in other words, the individual's observable traits. 

The GA model converts genotype to phenotype by calling the function `to_phenotype` with the genotype as an argument. The cell below shows the default code that the GA model uses to produce phenotypes. By default, the phenotype is exactly the same as the genotype — not very interesting!

In [0]:
def to_phenotype(genotype):
    """Convert a genotype into a phenotype."""
    return genotype

Now let's see what it does. In the cell below we select the first individial of the population and use the function `to_photype` to convert its genotype to phenotype.

In [7]:
# select the first individual
individual = ga.population[0]
# convert to phenotype
phenotype = ga.to_phenotype(individual)
# veiw phenotype
phenotype

[0, 1, 0, 1, 1, 0, 1, 0, 1, 0]

## 3. Fitness function
Next, the GA evaluates the fitness of each individual in the population. The fitness scores determines the likelihood that an individual will be chosen to reproduce and pass on its genetic information. The fitness function is what guides our evolution process, and by defining different fitness functions we can guide our evolution process towards different goals.  

**Important**: the fitness function operates on the  phenotype representation, not genotype. Fitness scores are a number between `0` and `1`, in which `0` represents **not** fit and `1` represents **very** fit.

The GA model evalautes fitness of an individual by calling the function `fitness_func` with an individual's phenotype as the argument. The cell below shows the default code that the GA model uses. By default, `fitness_func` counts the number of `1`'s in the phenotype representation, and then divides by the total number of elements. The more `1`'s the higher the score. Don't worry about understanding the code in this cell.

In [0]:
def fitness_func(phenotype):
    """Evaluate the fitness of an individual."""
    return sum(phenotype) / float(len(phenotype))

Now let's see what it does. In the cell below evalute the fitness of the first individual, which was selected and converted to phenotype in the previous step.

In [9]:
# evalute fitness of individual 0
fitness = ga.fitness_func(phenotype)
# view fitness
fitness

0.5

## 4. Construct mating pool
After evaluating fitness, the GA constructs a mating pool. The mating pool is the collection of individuals that we will chose from when selecting parents to reproduce offspring for the next generation.

Individuals are added to the mating pool in numbers proportional to their fitness scores. The idea is that if an individual is particularly well fit, there will be more copies of that individual in the mating pool, and therefore a greater chance of it being chosen to reproduce.

The GA model calls the function `to_weight` to determine how many copies of an indivual to add to the mating pool. By default, the GA multiplies the fitness score by `100` and then adds `1`. The cell below shows the default code:

In [0]:
def to_weight(fitness, m=100, b=1):
    """Convert from fitness score to probability weighting"""
    return int(round(fitness*m + b))

Now let's see what it does. In the cell below we call the function `to_weight` to see the weight of the first individual.

In [11]:
# convert fitness to weight
weight = ga.to_weight(fitness)
# view weight
weight

51

## 5. Reproduce
Next, the GA generates a new population based on the fitness scores of the previous generation. Pairs of individuals are chosen from the mating pool and combined to produce offspring. Each two parents produces two new offspring. Offspring are produced from parents by the means of a genetic operator called *crossover reproduction*.

In crossover reproducion, two parents genotypes are split and recombined to produce two new children genotypes. First, a random split point is selected. Then, the first child gets the first part of parent 1 and the second part of parent 2, and the second child gets the second part of parent 1 and the first part of parent 2.

**Important**: crossover reproduction operates on genotypes (not phenotpyes!) and produces new genotypes (not phenotypes!)

![crossover reproduction](https://raw.githubusercontent.com/davidkant/aim80L/master/resources/img/GA-crossover.png)

**figure:** crossover reproduction

The GA model calls the function `reproduce` and passes two parents as arguments. The cell below shows the default code:

In [0]:
def reproduce(parent1, parent2):
    """Generate offspring using random crossover."""

    # random crossover point
    crossover = random.randrange(0, len(parent1))

    # construct children
    child1 = parent1[0:crossover] + parent2[crossover:]
    child2 = parent2[0:crossover] + parent1[crossover:]

    # return children
    return child1, child2

Now let's see what it does. In the cell below we select two parents from the current population and call the function `reproduce` to produce two new children.

In [13]:
# select parents
parent1 = ga.population[0]
parent2 = ga.population[1]

# print parent genotypes
print 'parent1: {0}'.format(parent1)
print 'parent2: {0}'.format(parent2)

# reproduce
print 'children:'
ga.reproduce(parent1, parent2)

parent1: [0, 1, 0, 1, 1, 0, 1, 0, 1, 0]
parent2: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
children:


([0, 1, 0, 1, 0, 1, 0, 1, 0, 1], [0, 1, 0, 1, 1, 0, 1, 0, 1, 0])

## 6. Mutate
The final step in producing a new generation is  mutation. After reproducing a new generatio, we mutate a select few members of the population.

Mutation works by selecting a random element (called chromosome) of the genotype and "flipping" its value, which means subtracting it from 1. 

![crossover reproduction](https://raw.githubusercontent.com/davidkant/aim80L/master/resources/img/GA-mutation.png)

**figure:** mutation

For each member of the new population, the GA model calls the function `mutate`. The function first determines whether or not to mutate the individual. If chosen for mutation, the function selects a random chromosome and flips its value. The cell below shows the default code:

In [0]:
def mutate(genotype, mutation_prob=0.01):
    """Maybe mutate an individual."""
    
    # do we mutate this individual?
    if random.random() <= mutation_prob:
      
        # select a random chromosome
        gene_index = random.randrange(len(genotype))
        
        # flip its value
        genotype[gene_index] = 1 - genotype[gene_index]
        
    return genotype 

Now let's see what it does. In the cell below we mutate an individual.

In [15]:
print individual
ga.mutate(individual, mutation_prob=1)

[0, 1, 0, 1, 1, 0, 1, 0, 1, 0]
-> muuuuutating individual [0, 1, 0, 1, 1, 0, 1, 0, 1, 0]


[1, 1, 0, 1, 1, 0, 1, 0, 1, 0]

## And... repeat!
The GA repeates steps 2 through 6 to evolve many generations of populations. Hopefully the individuals become more fit as we evolve!

# Let's evolve something!
All of the above code is packaged inside the `GeneticAlgorithm()` model such that we only have to call the function `evolve()` to run it. The function `evolve()` takes a number of arguments:

- `iters` is the number of generations to evolve
- `population_size` is the number of individuals in each population
- `init_pop` is a boolean that determines whether or not we initialize a random population or start from where we left off. 
- `mutation_prob` is the probability that an individual will be mutated

In the cell below we evolve 10 generations of 100 individuals each. Try changing the number of generations and population size and see what happens! Try changing the mutation probability, too!

In [16]:
# create a new genetic algorithm
ga = mai.genalg.GeneticAlgorithm()

# evolve 10 times
gen = ga.evolve(iters=10,
                population_size=100,
                init_pop=True,
                mutation_prob=0.01)

# view the current population
print '\ncurrent population:\n'
gen

-> muuuuutating individual [1, 1, 0, 0, 1, 1, 0, 1, 1, 1]
-> muuuuutating individual [0, 1, 0, 1, 1, 0, 0, 0, 1, 1]
-> muuuuutating individual [0, 1, 0, 0, 0, 0, 1, 0, 1, 0]
-> muuuuutating individual [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
-> muuuuutating individual [1, 0, 1, 0, 0, 1, 1, 1, 1, 1]
-> muuuuutating individual [1, 1, 0, 1, 0, 1, 1, 1, 1, 1]
-> muuuuutating individual [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
-> muuuuutating individual [1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
-> muuuuutating individual [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

current population:



([1, 1, 1, 1, 0, 1, 0, 1, 1, 1],
 [1, 0, 0, 1, 1, 0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 0, 1, 1, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 0, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 0, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [0, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 0, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
 [1, 1, 0, 1, 0, 1, 1, 0, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1,

Not a terribly exciting evolution, but in the next notebook we'll learn how to customize the GA to our own musical environments by replacing the default functions.